In [1]:
# df_adjOHLCV has data only for NYSE trading days, no weekend data
# df_adjOHLCV includes data for weekends when BTC trades
# read symbols in file to list syms_in_file
# download OHLCV data for symbols in syms_in_file
# drop symbols with all NaN in OHLCV columns from df
# rename column names from ['Open', ..., 'Volume'] to ['open', ..., 'volume']
# drop weekend data by reindex to date index of index_symbol
# pickled df_adjOHLCV
# pickled df_adjOHLCV
# pickled symbols_df_adjOHLCV
# create df_symbols_close, sort df by symbols
# pickled df_symbols_close

![alt text](image_url)

In [2]:
# https://stackoverflow.com/questions/67690778/how-to-detect-failed-downloads-using-yfinance
import yfinance as yf
import yfinance.shared as shared
import time
import pandas as pd
# from datetime import date, timedelta, datetime
from myUtils import pickle_dump, pickle_load, read_symbols_file # NOQA
from myUtils import drop_symbols_all_NaN, chunked_list # NOQA
from myUtils import yf_download_AdjOHLCV_noAutoAdj
verbose = False  # True prints more output

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
# filename_symbols = path_data_dump + 'vg_symbols_4chars_max_clean.csv'  # symbols text file
filename_symbols = path_data_dump + 'my_symbols.csv'  # symbols text file

filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV

In [3]:
# function to adjust OHLCV
# https://github.com/ranaroussi/yfinance/issues/687
def auto_adjust(data):
    df = data.copy()
    ratio = df["Close"] / df["Adj Close"]
    df["Adj Open"] = df["Open"] / ratio
    df["Adj High"] = df["High"] / ratio
    df["Adj Low"] = df["Low"] / ratio

    df.drop(
        ["Open", "High", "Low", "Close"],
        axis=1, inplace=True)

    df.rename(columns={
        "Adj Open": "Open", "Adj High": "High",
        "Adj Low": "Low", "Adj Close": "Close"
    }, inplace=True)

    # df = df[["Open", "High", "Low", "Close", "Volume"]]
    return df[["Open", "High", "Low", "Close", "Volume"]]

def close_vs_Yahoo (symbols, df):
  for symbol in symbols:
    s = df.iloc[-250]
    
    sDate = s.name.strftime('%Y-%m-%d')
    sClose = s[symbol].Close

    df_sym = yf.Ticker(symbol).history(period='2y')
    # 10:45PM got Adj Close
    yhClose = df_sym.loc[sDate]['Close']

    abs_pct_diff = abs(1 - sClose/yhClose)*100
    print(f'symbol:  {symbol:>4}   Date: {sDate:13}df_Close: {sClose:>10.5f} \
    Yahoo_Close: {yhClose:>10.5f}   %_dif_Close: {abs_pct_diff:>7.5f}')
    if abs_pct_diff > .0001:
      msg_err = f'{symbol}  %_dif_Close > .0001'
      # raise SystemExit(msg_err)
      print(msg_err)  # even if discrepancy, the relative relationship of OHLCV in df_close matches Yahoo_Close
    if symbol == symbols[-1]:
      msg_done = f"No errors found.  Symbols' 'Adjusted Close' matched Yahoo symbols' Close "
      print(msg_done)


In [4]:
# Stop if last date in df_adjOHLCV is the same as Yahoo download
index_symbol = "XOM"  
df_XOM = yf.download(index_symbol)
download_last_date = df_XOM.index[-1].strftime('%Y-%m-%d')
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
print(f'download_last_date: {download_last_date}')
df = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)
df_adjOHLCV_last_date = df.index[-1].strftime('%Y-%m-%d')
print(f'df_adjOHLCV_last_date: {df_adjOHLCV_last_date}')

##########  Allow Repeated Download on the Same Day  ###########################################
# if download_last_date == df_adjOHLCV_last_date:
#   msg_stop = f"df_adjOHLCV is up-to-date,\nlast date from df_adjOHLCV is {df_adjOHLCV_last_date},\nlast date from Yahoo's {index_symbol} download is {download_last_date}"
#   raise SystemExit(msg_stop )
######################################################

[*********************100%***********************]  1 of 1 completed
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV
download_last_date: 2023-09-25
df_adjOHLCV_last_date: 2023-09-22


In [5]:
# verify df test_symbols' close against Yahoo
# even if discrepancy, the relative relationship of OHLCV in df_close matches Yahoo_Close
test_symbols = ['A', 'LMT', 'SPY']
_df_list=[]
_df = yf_download_AdjOHLCV_noAutoAdj(test_symbols, verbose=False)
_df_list.append(_df)
_df = pd.concat(_df_list, axis=1)
# get unique symbols from column name
_l_symbols = list(set(_df.columns.get_level_values(0)))
_l_symbols.sort()

_df_adj_list=[]
# adjust OHLC using 'Adj Close'
for symbol in _l_symbols:
  _df1 = auto_adjust(_df[symbol])
  _df_adj_list.append(_df1)

_df_adj = pd.concat(_df_adj_list, axis=1)
_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
# create multilevel column names
_col_names = pd.MultiIndex.from_product([_l_symbols, _cols])
_df_adj.columns = _col_names

close_vs_Yahoo(test_symbols, _df_adj)

[*********************100%***********************]  3 of 3 completed
symbol:     A   Date: 2022-09-27   df_Close:  120.79128     Yahoo_Close:  120.79129   %_dif_Close: 0.00001
symbol:   LMT   Date: 2022-09-27   df_Close:  389.53317     Yahoo_Close:  389.53320   %_dif_Close: 0.00001
symbol:   SPY   Date: 2022-09-27   df_Close:  357.74826     Yahoo_Close:  357.74826   %_dif_Close: 0.00000
No errors found.  Symbols' 'Adjusted Close' matched Yahoo symbols' Close 


In [6]:
# read symbols in file to a list and convert to chunks
#  each chunk must have more than 1 symbol, otherwise, symbol won't appear as column name
symbols_in_file = read_symbols_file(filename_symbols)
print(f'symbols in file: {len(symbols_in_file)}')
symbols_add = ['SPY', 'VGT', 'FTEC', 'QQQ', 'VWO', 'VTV', 'BNDX', 'USO', 'JJC', 'BCI', 'NG', 'GBTC', 'BTC-USD', 'ETH-USD', 'SGOV', 'GLD']
print(f'symbols added:   {len(symbols_add)}')
symbols = symbols_in_file + symbols_add
# https://stackoverflow.com/questions/12897374/get-unique-values-from-a-list-in-python
symbols = list(set(symbols))  # keep unique symbols
symbols.sort()  # sort unique symbols in alphabetical order
print(f'unique symbols:  {len(symbols)}')
symbols_chunks = chunked_list(symbols, 400)  # e.g. [['A', 'BB', ...], ['CC', 'DD', ...], ..., ['Z', 'ZWS', ...]]
for chunk in symbols_chunks:
  if len(chunk) == 1:
    raise SystemExit(f'ERROR, Must be more than 1 symbol in each chunk\nsymbols in chunk: {chunk}')

symbols in file: 1578
symbols added:   16
unique symbols:  1587


In [7]:
df_list=[]
symbols_download_err = []
# took 24 minutes to download 1917 symbols without error caused by Yahoo
# for i, symbols in enumerate(symbols_chunks):
for i, symbols in enumerate(symbols_chunks):
  df = yf_download_AdjOHLCV_noAutoAdj(symbols, verbose=False)  
  symbols_download_err.append(list(shared._ERRORS.keys()))
  # print(f'symbols_download_err: {symbols_download_err}')
  print(f'\nsymbols_download_err: {symbols_download_err}')  
  df_list.append(df)
  # pause between download
  if i < len(symbols_chunks) - 1 :  # skip pause after last download
    print(f'downloaded symbols from chuck {i}, sleep start')
    # sleep 78(18m 25s)
    time.sleep(78)
    print(f'downloaded symbols from chuck {i}, sleep ends')
  else:
    print(f'downloaded symbols from all chucks')

print(f'symbols_download_err: {symbols_download_err}')

[*********************100%***********************]  400 of 400 completed

4 Failed downloads:
- AAWW: No data found, symbol may be delisted
- DCP: No data found, symbol may be delisted
- ABB: No data found, symbol may be delisted
- ATCO: No data found, symbol may be delisted

symbols_download_err: [['AAWW', 'DCP', 'ABB', 'ATCO']]
downloaded symbols from chuck 0, sleep start
downloaded symbols from chuck 0, sleep ends
[*********************100%***********************]  400 of 400 completed

4 Failed downloads:
- IAA: No data found, symbol may be delisted
- FRC: No data found, symbol may be delisted
- ISEE: No data found, symbol may be delisted
- FISV: No data found, symbol may be delisted

symbols_download_err: [['AAWW', 'DCP', 'ABB', 'ATCO'], ['IAA', 'FRC', 'ISEE', 'FISV']]
downloaded symbols from chuck 1, sleep start
downloaded symbols from chuck 1, sleep ends
[*********************100%***********************]  400 of 400 completed

4 Failed downloads:
- LSI: No data found, symbol may

In [8]:
# flatten symbols_download_err which is a list-of-lists
l_symbols_err = [val for sublist in symbols_download_err for val in sublist]
pickle_dump(l_symbols_err, path_data_dump, 'l_symbols_err')
l_symbols_err

['AAWW',
 'DCP',
 'ABB',
 'ATCO',
 'IAA',
 'FRC',
 'ISEE',
 'FISV',
 'LSI',
 'PKI',
 'RE',
 'MGI',
 'SJR',
 'TA',
 'ROCC']

In [9]:
# if l_symbols_err:  # re-download symbols with download error 
#   symbols_download_err = []
#   # took 24 minutes to download 1917 symbols without error caused by Yahoo
#   # for i, symbols in enumerate(symbols_chunks):
#   for i, symbols in enumerate(l_symbols_err):
#     df = yf_download_AdjOHLCV_noAutoAdj(symbols, verbose=False)  
#     symbols_download_err.append(list(shared._ERRORS.keys()))
#     # print(f'symbols_download_err: {symbols_download_err}')
#     print(f'\nsymbols_download_err: {symbols_download_err}')  
#     df_list.append(df)
#     # pause between download
#     if i < len(symbols_chunks) - 1 :  # skip pause after last download
#       print(f'downloaded symbols from chuck {i}, sleep start')
#       # sleep 78(18m 25s)
#       time.sleep(78)
#       print(f'downloaded symbols from chuck {i}, sleep ends')
#     else:
#       print(f'downloaded symbols from all chucks')

#   print(f'symbols_download_err: {symbols_download_err}')

In [10]:
df = pd.concat(df_list, axis=1)
if l_symbols_err:  # if list is not empty, drop symbols with errors
  print(f'l_symbols_err is not empty: {l_symbols_err}')  
  df.drop(l_symbols_err, axis=1, level=0, inplace=True)  # drop symbols with errors

l_symbols = list(set(df.columns.get_level_values(0)))  # get unique symbols
l_symbols.sort()
df = df[l_symbols]  # sort columns by list
print(f"Full path to pickled df_OHLCVA_downloaded:  {path_data_dump}{filename_pickled_df_OHLCVA_downloaded}")
pickle_dump(df, path_data_dump, filename_pickled_df_OHLCVA_downloaded)  # save OHLCA data

l_symbols_err is not empty: ['AAWW', 'DCP', 'ABB', 'ATCO', 'IAA', 'FRC', 'ISEE', 'FISV', 'LSI', 'PKI', 'RE', 'MGI', 'SJR', 'TA', 'ROCC']
Full path to pickled df_OHLCVA_downloaded:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCVA_downloaded 


In [11]:
# adjust OHLC
df_adj_list=[]
for symbol in l_symbols:
  _df = auto_adjust(df[symbol])
  df_adj_list.append(_df)

df_adj = pd.concat(df_adj_list, axis=1)
cols = ['Open', 'High', 'Low', 'Close', 'Volume']
col_names = pd.MultiIndex.from_product([l_symbols, cols])  # create multilevel column names
df_adj.columns = col_names  # reindex columns

print(f"Full path to pickled df_adjOHLCV: {path_data_dump}{filename_pickled_df_adjOHLCV}")
pickle_dump(df_adj, path_data_dump, filename_pickled_df_adjOHLCV)  # save adjusted OHLCV data

Full path to pickled df_adjOHLCV: C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV


In [12]:
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV)

In [13]:
# drop symbols with all NaN in OHLCV columns from df
df_adjOHLCV, symbols_OHLCV, symbols_dropped = drop_symbols_all_NaN(df_adjOHLCV, verbose)
print(f'symbols with all NaN dropped from df_adjOHLCV: {symbols_dropped}')

Symbol's OHLCV are all NaN: []
symbols with all NaN dropped from df_adjOHLCV: []


In [14]:
# rename columns OHLCV *ONLY AFTER* dropping symbols with all NaN from df,
#   symbols with all NaN has an added AdjClose column and will cause errors  
#  rename column names from ['Open', ..., 'Volume'] to ['open', ..., 'volume']
#  .remove_unused_levels() prevents ValueError
#   e.g ValueError: On level 1, code max (5) >= length of level (5). NOTE: this index is in an inconsistent state
# The error may be caused by removing symbols from the dataframe with all NaN in OHLCV columns
df_adjOHLCV.columns = df_adjOHLCV.columns.remove_unused_levels()
# # set_levels reorders df columns in alphabetical order, so the list of column names also needs to be in alphabetical order
# df_adjOHLCV.columns = df_adjOHLCV.columns.set_levels(['close', 'high', 'low', 'open', 'volume'], level=1)

In [15]:
# drop weekend data by re-indexing to date-index of index_symbol
myNaN = float('nan')
# use Exxon's date as proxy for NYSE trading dates
df_adjOHLCV = df_adjOHLCV.reindex(df_XOM.index, fill_value=myNaN)

In [16]:
# pickle df_adjOHLCV and symbols
# print(f"Full path to pickled df_adjOHLCV_downloaded:  {path_data_dump}{filename_pickled_df_adjOHLCV_downloaded}")
# pickle_dump(df_adjOHLCV_downloaded, path_data_dump, filename_pickled_df_adjOHLCV_downloaded, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
pickle_dump(df_adjOHLCV, path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)
print(f"Full path to pickled symbols_df_adjOHLCV:  {path_data_dump}{filename_pickled_symbols_df_adjOHLCV}")
pickle_dump(symbols_OHLCV, path_data_dump, filename_pickled_symbols_df_adjOHLCV, verbose=verbose)

Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV
Full path to pickled symbols_df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/symbols_df_adjOHLCV


In [17]:
symbols_OHLCV = list(set([i[0] for i in list(df_adjOHLCV)]))
df_symbols_close = df_adjOHLCV.xs('Close', level=1, axis=1)
pickle_dump(df_symbols_close, path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)

In [18]:
# retrieve pickled files

print(f"Full path to pickled df_OHLCVA_downloaded:  {path_data_dump}{filename_pickled_df_OHLCVA_downloaded}")
df_OHLCVA_downloaded = pickle_load(path_data_dump, filename_pickled_df_OHLCVA_downloaded, verbose=verbose)

print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)

print(f"Full path to pickled symbols_df_adjOHLCV:  {path_data_dump}{filename_pickled_symbols_df_adjOHLCV}")
symbols_df = pickle_load(path_data_dump, filename_pickled_symbols_df_adjOHLCV, verbose=verbose)

Full path to pickled df_OHLCVA_downloaded:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCVA_downloaded 
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV
Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled symbols_df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/symbols_df_adjOHLCV


In [19]:
print(f'df_OHLCVA_downloaded.info():\n{df_OHLCVA_downloaded.info()}\n')
print(f'df_adjOHLCV.info():\n{df_adjOHLCV.info()}\n')
print(f'df_close.info():\n{df_close.info()}\n')
print(f'len(symbols_df):\n{len(symbols_df)}\n')

<class 'pandas.core.frame.DataFrame'>
Index: 16565 entries, 1962-01-02 00:00:00 to 2023-09-26 00:00:00
Columns: 9432 entries, ('A', 'Open') to ('ZWS', 'Volume')
dtypes: float64(9432)
memory usage: 1.2+ GB
df_OHLCVA_downloaded.info():
None

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15539 entries, 1962-01-02 to 2023-09-25
Columns: 7860 entries, ('A', 'Open') to ('ZWS', 'Volume')
dtypes: float64(7860)
memory usage: 931.9 MB
df_adjOHLCV.info():
None

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15539 entries, 1962-01-02 to 2023-09-25
Columns: 1572 entries, A to ZWS
dtypes: float64(1572)
memory usage: 186.5 MB
df_close.info():
None

len(symbols_df):
1572



In [20]:
count_symbols = len(symbols_df)
count_df_close_cols = len(df_close.columns)
count_cols_df_OHLCVA_downloaded = len(df_OHLCVA_downloaded.columns)
count_cols_df_adjOHLCV = len(df_adjOHLCV.columns)
if count_symbols != count_df_close_cols:
  raise SystemExit(f'symbol count: {count_symbols} does not equal df_close column count: {count_df_close_cols}')
if count_symbols * 6 != count_cols_df_OHLCVA_downloaded:
  raise SystemExit(f'6 x symbol count: {6 * count_symbols} does not equal df_OHLCVA_downloaded column count: {count_cols_df_OHLCVA_downloaded}')
if count_symbols * 5 != count_cols_df_adjOHLCV:
  raise SystemExit(f'5 x symbol count: {5 * count_symbols} does not equal df_adjOHLCV column count: {count_cols_df_adjOHLCV}')
print(f'No error found, Column counts in df_close, df_OHLCVA, df_adjOHLCV matched symbol count')


No error found, Column counts in df_close, df_OHLCVA, df_adjOHLCV matched symbol count


In [21]:
print(f'count_symbols: {count_symbols}')
print(f'count_df_close_cols: {count_df_close_cols}')
print(f'count_cols_df_OHLCVA_downloaded: {count_cols_df_OHLCVA_downloaded}')
print(f'count_cols_df_adjOHLCV: {count_cols_df_adjOHLCV}')

count_symbols: 1572
count_df_close_cols: 1572
count_cols_df_OHLCVA_downloaded: 9432
count_cols_df_adjOHLCV: 7860


In [22]:
close_vs_Yahoo(test_symbols, df_adjOHLCV)

symbol:     A   Date: 2022-09-27   df_Close:  120.79129     Yahoo_Close:  120.79129   %_dif_Close: 0.00000
symbol:   LMT   Date: 2022-09-27   df_Close:  389.53317     Yahoo_Close:  389.53317   %_dif_Close: 0.00000
symbol:   SPY   Date: 2022-09-27   df_Close:  357.74826     Yahoo_Close:  357.74826   %_dif_Close: 0.00000
No errors found.  Symbols' 'Adjusted Close' matched Yahoo symbols' Close 


In [23]:
_sym = 'NOC'
s_start, s_end = -252, -248
df_adjOHLCV[_sym].iloc[s_start:s_end]
df_OHLCVA_downloaded[_sym].iloc[s_start:s_end]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-01-18 00:00:00,453.730011,454.079987,445.510010,446.059998,440.607788,1927400.0
2023-01-19 00:00:00,448.149994,453.010010,442.209991,442.309998,436.903625,1376600.0
2023-01-20 00:00:00,445.989990,452.299988,440.089996,450.760010,445.250366,2130900.0
2023-01-21 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df_adjOHLCV.tail()

A                                                    AA  \
                  Open        High         Low       Close     Volume   Open   
Date                                                                           
2023-09-19  112.370003  113.230003  111.720001  112.309998  2222400.0  28.83   
2023-09-20  113.080002  113.470001  112.169998  112.470001  1744900.0  28.65   
2023-09-21  112.029999  112.199997  109.500000  109.730003  1327900.0  28.68   
2023-09-22  109.730003  112.160004  109.110001  111.410004  2050500.0  28.27   
2023-09-25  110.760002  112.029999  110.529999  111.940002   987400.0  27.75   

                                                        ...       ZUMZ  \
                 High        Low      Close     Volume  ...       Open   
Date                                                    ...              
2023-09-19  28.989000  27.834999  28.340000  5151500.0  ...  16.510000   
2023-09-20  29.790001  28.549999  29.379999  8084100.0  ...  16.930000   
2023-09-21  28.680000  27.860001  27.870001  5487500.0  ...  17.430000   
2023-09-22  28.596001  28.049999  28.350000  5095000.0  ...  17.309999   
2023-09-25  27.770000  26.340000  26.629999  9298000.0  ...  17.299999   

                                                             ZWS             \
                 High        Low      Close    Volume       Open       High   
Date                                                                          
2023-09-19  17.120001  16.510000  16.950001  406200.0  28.120001  28.340000   
2023-09-20  17.719999  16.930000  17.670000  314600.0  28.260000  28.520000   
2023-09-21  17.590000  17.209999  17.270000  341000.0  27.760000  27.790001   
2023-09-22  17.530001  17.209999  17.410000  289500.0  27.600000  27.920000   
2023-09-25  17.860001  17.129999  17.809999  219600.0  27.490000  27.830000   

                                            
                  Low      Close    Volume  
Date                                        
2023-09-19  27.889999  28.139999  665000.0  
2023-09-20  27.959999  27.969999  576900.0  
2023-09-21  27.450001  27.480000  605500.0  
2023-09-22  27.490000  27.620001  675200.0  
2023-09-25  27.490000  27.740000  853400.0  

[5 rows x 7860 columns]

In [25]:
syms_def = ['NOC', 'HII', 'AVAV', 'LMT', 'CW', 'HEI']
_df = df_close[syms_def]
_df.tail()


,NOC,HII,AVAV,LMT,CW,HEI
Date,,,,,,
2023-09-19,435.970001,208.919998,111.870003,425.309998,205.259995,166.320007
2023-09-20,440.179993,205.580002,112.300003,426.950012,203.380005,165.740005
2023-09-21,437.109985,202.500000,111.349998,417.540009,199.229996,160.820007
2023-09-22,424.899994,201.080002,112.330002,413.649994,197.789993,159.669998
2023-09-25,425.570007,201.330002,112.089996,412.899994,198.699997,159.500000


In [26]:
syms_LNG = ['LNG', 'CHK', 'GLNG']
_df = df_close[syms_LNG]
_df.tail()

,LNG,CHK,GLNG
Date,,,
2023-09-19,161.610001,84.669998,23.360001
2023-09-20,161.470001,83.809998,23.230000
2023-09-21,158.850006,82.699997,23.070000
2023-09-22,159.919998,82.070000,23.129999
2023-09-25,164.139999,83.089996,23.450001


In [27]:
_df = df_close[symbols_add]
_df.tail()

,SPY,VGT,FTEC,QQQ,VWO,VTV,BNDX,USO,JJC,BCI,NG,GBTC,BTC-USD,ETH-USD,SGOV,GLD
Date,,,,,,,,,,,,,,,,
2023-09-19,442.709991,427.309998,126.550003,369.869995,39.950001,142.310745,48.110001,81.500000,NaN,21.639999,4.11,19.840000,27211.117188,1643.544678,100.480003,179.220001
2023-09-20,438.640015,421.019989,124.610001,364.540009,39.830002,141.953003,48.180000,80.139999,NaN,21.490000,4.39,19.420000,27132.007812,1622.890625,100.510002,179.350006
2023-09-21,431.390015,414.040009,122.660004,357.859985,39.189999,140.169998,48.130001,80.410004,NaN,21.320000,4.36,19.000000,26567.632812,1584.307007,100.540001,178.050003
2023-09-22,430.420013,415.190002,122.989998,357.910004,39.650002,139.690002,48.160000,80.629997,NaN,21.410000,4.35,18.900000,26579.568359,1593.268311,100.559998,178.619995
2023-09-25,432.230011,416.839996,123.440002,359.609985,39.500000,140.119995,47.970001,80.389999,NaN,21.309999,4.27,18.450001,NaN,NaN,100.580002,177.699997


In [28]:
# syms_def = ['NOC', 'HII', 'AVAV', 'LMT', 'CW', 'HEI']
s_start, s_end = -252, -247
_df = df_close[test_symbols].iloc[s_start:s_end]
_df

,A,LMT,SPY
Date,,,
2022-09-23,122.648705,402.522827,362.247437
2022-09-26,121.486572,397.241211,358.663818
2022-09-27,120.791290,389.533173,357.748260
2022-09-28,122.887093,389.786530,364.787415
2022-09-29,121.913673,379.837250,357.167419
